In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df = pd.read_csv('../input/competitive-data-science-predict-future-sales/sales_train.csv')
sample = pd.read_csv('../input/competitive-data-science-predict-future-sales/sample_submission.csv')

In [ ]:
df.head()

In [ ]:
sample.head()

In [ ]:
items = pd.read_csv('../input/competitive-data-science-predict-future-sales/items.csv')
items_category = pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
shops = pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')

In [ ]:
items.head()

In [ ]:
items.shape

In [ ]:
items_category.shape

In [ ]:
items.head(5)

In [ ]:
items = items.drop(columns = ['item_name','item_name'])

In [ ]:
items_category.head()

In [ ]:
shops.head()

In [ ]:
shops.shape

In [ ]:
category = []
for i in df['item_id']:
    category.append(items['item_category_id'][i])

In [ ]:
print(category[0:20])

In [ ]:
items.iloc[22154,:]

In [ ]:
df['item_category_id'] = category

In [ ]:
df.head()

In [ ]:
data = df[df['item_cnt_day']<=50]

In [ ]:
data = data[data['item_cnt_day']>-3]

In [ ]:
data.shape

In [ ]:
df.shape

In [ ]:
len(data['item_cnt_day'].unique())

In [ ]:
data_train = data.drop(columns = ['item_cnt_day','item_id','date'])

In [ ]:
target = data['item_cnt_day']

In [ ]:
print(target.value_counts())

In [ ]:
# temp = data.value_counts()
# import matplotlib.pyplot as plt
# plt.plot(temp[:])

In [ ]:
data.shape

In [ ]:
data.head()

In [ ]:
data.drop(columns = ['date','item_id'],inplace = True)

In [ ]:
date_block = []
for i in data['date_block_num']:
    date_block.append(i%12)
        

In [ ]:
data['date_block_engineered'] = date_block

In [ ]:
data['date_block_engineered'].unique()

In [ ]:
data.drop(columns = ['date_block_num'],inplace = True)

In [ ]:
data.head()

In [ ]:
import tensorflow.keras as keras

In [ ]:
labels = data['item_cnt_day']

In [ ]:
len(labels.unique())

In [ ]:
# labels = pd.get_dummies(labels)

In [ ]:
labels.head()

In [ ]:
labels.shape

In [ ]:
data.head()

In [ ]:
data = data.drop(columns = ['item_cnt_day'])

In [ ]:
data.shape

In [ ]:
labels.shape

In [ ]:
label = labels.transpose()

In [ ]:
keras.backend.clear_session()
model = keras.models.Sequential([
    keras.layers.Dense(3,input_dim = 2,activation = 'relu'),
    keras.layers.Dense(1,activation = 'relu')
])
early = keras.callbacks.EarlyStopping(patience = 5)
model_check = keras.callbacks.ModelCheckpoint('model.h5',save_best_only = True)
model.compile(loss = 'mse',optimizer = 'adam',metrics = ['accuracy'])


In [ ]:
data.head()

In [ ]:
data_x = data.drop(columns = ['date_block_engineered','item_price'])

In [ ]:
data_x.head()

In [ ]:
model.fit(data_x,labels,epochs = 500,validation_split = 0.2,callbacks = [early,model_check],batch_size = 64)

In [ ]:
test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')

In [ ]:
item_category = []
for i in test['item_id']:
    item_category.append(items['item_category_id'][i])

In [ ]:
test['item_category_id'] = item_category

In [ ]:
test.head()

In [ ]:
# df_sorted = df.sort_values(['item_id'])

In [ ]:
# df_sorted.head()

In [ ]:
test = test.drop(columns = ['ID','item_id'])

In [ ]:
predictions = model.predict(test)

In [ ]:
predictions = pd.DataFrame(predictions)

In [ ]:
predictions.head()

In [ ]:
pred = []
for i in predictions[0]:
    pred.append(round(i))

In [ ]:
predictions['item_cnt_day'] = pred

In [ ]:
predictions.head()

In [ ]:
test = pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')

In [ ]:
predictions['ID'] = test['ID']

In [ ]:
predictions.head()

In [ ]:
predictions = predictions.drop(columns = [0])

In [ ]:
predictions.head()

In [ ]:
predictions.to_csv('Submit_1.csv',index = False)

In [ ]:
predictions.columns = ['item_cnt_month','ID']

In [ ]:
predictions.to_csv('Submit_2.csv',index = False)